## Accelerate Inference: Neural Network Pruning

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.14.0


In [ ]:
# untar
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [ ]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [ ]:
# you can use the default hyper-parameters for training,
# val accuracy ~72% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50,
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50
703/703 [==============================] - 17s 6ms/step - loss: 1.4935 - accuracy: 0.3226 - val_loss: 1.3848 - val_accuracy: 0.4055
Epoch 2/50
703/703 [==============================] - 4s 6ms/step - loss: 1.3212 - accuracy: 0.4376 - val_loss: 1.2350 - val_accuracy: 0.4784
Epoch 3/50
703/703 [==============================] - 5s 6ms/step - loss: 1.2646 - accuracy: 0.4692 - val_loss: 1.2079 - val_accuracy: 0.4947
Epoch 4/50
703/703 [==============================] - 4s 6ms/step - loss: 1.2265 - accuracy: 0.4890 - val_loss: 1.1803 - val_accuracy: 0.5097
Epoch 5/50
703/703 [==============================] - 4s 6ms/step - loss: 1.1901 - accuracy: 0.5096 - val_loss: 1.1362 - val_accuracy: 0.5323
Epoch 6/50
703/703 [==============================] - 4s 6ms/step - loss: 1.1583 - accuracy: 0.5265 - val_loss: 1.1153 - val_accuracy: 0.5374
Epoch 7/50
703/703 [==============================] - 5s 6ms/step - loss: 1.1331 - accuracy: 0.5413 - val_loss: 1.0869 - val_accuracy: 0.5640
Epoch

In [ ]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 8ms/step - loss: 0.7300 - accuracy: 0.7184


In [ ]:
# perform pruning here

# get the weights
weights = model.get_weights()

# you can use set_weights() to set some weights to zero, e.g.,

weights[7][:10]=0
model.set_weights(weights)


In [ ]:
print(type(weights))
print([weights[i].shape for i in range(len(weights))])

<class 'list'>
[(3, 3, 3, 32), (32,), (3, 3, 32, 32), (32,), (3, 3, 32, 64), (64,), (3, 3, 64, 64), (64,), (1024, 512), (512,), (512, 5), (5,)]


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [ ]:
# evaluate again to see how the accuracy changes
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 4ms/step - loss: 0.7264 - accuracy: 0.7232


In [ ]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model.save_weights("my_model_weights.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>